<a href="https://colab.research.google.com/github/matheusomendonca/pos-ia-ds/blob/master/Classes/Data%20visualization/%5BProfessor%5D_Data_visualization_Covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualização dos dados disponibilizados pelo Imperial College sobre a pesquisa do impacto do covid-19 em diferentes cenários de mitigação de risco

O primeiro passo é importar as bibliotecas de visualização necessárias. Iremos trabalhar com umas bibliotecas básicas:

- [matplotlib](https://matplotlib.org/): biblioteca básica de visualização em Python;
- [seaborn](https://seaborn.pydata.org/): biblioteca de visualização estatística escrita baseada no matplotlib.

E outras um pouco mais sofisticadas...
- [missingno](https://github.com/ResidentMario/missingno): biblioteca usada para visualização de dados faltantes;
- [plotly](https://plotly.com/): biblioteca para visualização interativa.

In [1]:
# bibliotecas básicas
import pandas as pd
import numpy as np

# bibliotecas de visualização
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import missingno as msno #! pip install missingno

A seguir, iremos realizar definir o tamanho padrão das imagens ao longo deste notebook.

In [ ]:
# configuração do plot
figsize = (10, 7)
matplotlib.rcParams['figure.figsize'] = figsize

## Leitura dos dados

Iremos realizar a visualização dos dados disponibilizados pelo Imperial College no dia 26/03/2020 sobre a [pesquisa](https://www.imperial.ac.uk/media/imperial-college/medicine/sph/ide/gida-fellowships/Imperial-College-COVID19-Global-Impact-26-03-2020.pdf) que simula o impacto global de diferentes estratégias de mitigação do avanço do corona vírus.

A [base de dados](https://pycourse.s3.amazonaws.com/ImperialCollege-COVID19-scenarios.xlsx) é um arquivo excel com 4 worksheets:

1. Countries: dados básicos do [World Bank](https://www.worldbank.org/);
2. Mitigation: impactos do covid-19 para diferentes níveis de isolamento social;
3. Suppression: impactos do covid-19 para medidas de isolamento tomadas somente após um gatilho;
4. Dictionary: dicionário descritivo das variáveis.

O worksheet 4 é muito importante para o completo entendimento dos dados. A seguir está replicado o conteúdo deste worksheet.

**Mitigation Sheet**	
	
- **Country**:	Name of country or region
- **R0**:	Basic Reproduction number (average number of secondary infection by a typical infection in an unconstrained epidemic and wholly susceptible population)
- **Strategy**:
	- "Unmitigated" = no intervention; 
	- "Social distancing whole population" = optimal outcome when epidemic is mitigated interventions to limit contacts in the general population including social distancing; 
	- "Enhanced social distance of elderly"= optimal outcome when epidemic is mitigated through interventions to limit contacts in the general population including social distancing, alongside enhanced social distancing of over 70s (modelled as a 60% reduction in contact rate); 
- **Social_distance**:	Percentage reduction in contacts in the general population
	- For "Unmitigated" this is 0
	- For "Social distancing whole population" this reduction applies to the whole population
	- For "Enhanced social distancing of elderly" this applies to just individuals under 70. For individuals 70 and above we assume a 60% reduction.
- **total_pop**: Total population according to 2020 World Population Prospects (https://population.un.org/wpp/)
- **total_infected**: Final total population infected by the end of the pandemic
- total_death: Final total deaths by the end of the pandemic (assuming China-like age-severity profile and health system)
- **total_hospital**: Final total individuals requiring hospitalisation by the end of the pandemic (assuming China-like age-severity profile)
- **total_critical**: Final total individuals requiring critical care by the end of the pandemic (assuming China-like age-severity profile)

**Suppression Sheet**	
- **Country**:	Name of country or region
- **R0**:	Basic Reproduction number (average number of secondary infection by a typical infection in an unconstrained epidemic and - wholly susceptible population)
- **Strategy**
	- "Unmitigated" = no intervention
	- "0.2 deaths per million per week trigger" = suppression triggered when weekly death rate reaches this threshold
	- "1.6 deaths per million per week trigger" = suppression triggered when weekly death rate reaches this threshold
- **Social_distance**:	Proportional reduction in contact rate in general population modelled
- **total_pop**:	Total population according to 2020 World Population Prospects (https://population.un.org/wpp/)
- **total_infected**:	Final total population infected by the end of the pandemic. 
- **total_death**:	Final total deaths by the end of the pandemic (assuming China-like age-severity profile and health system). 
- **total_hospital**:	Final total individuals requiring hospitalisation by the end of the pandemic (assuming China-like age-severity profile)
- **peak_hospital_bed_demand**:	Number of hospital beds occupied at the peak of the epidemic (i.e. maximum demand/occupancy during the epidemic)
- **total_critical**:	Final total individuals requiring critical care by the end of the pandemic (assuming China-like age-severity profile)
- **peak_critical_bed_demand**:	Number of critical care beds occupied at the peak of the epidemic (i.e. maximum demand/occupancy during the epidemic)

Enfim, vamos realizar a leitura dos dados:

In [ ]:
# leitura dos dados
data = pd.ExcelFile("https://pycourse.s3.amazonaws.com/ImperialCollege-COVID19-scenarios.xlsx")

Como é um excel, precisamos ver as worksheets (abas) disponíveis...

In [ ]:
# worksheets disponíveis
print(f"Worksheets disponíveis: {data.sheet_names}")

No primeiro momento, iremos analisar indivualmente cada uma das 3 worksheets disponibilizadas.

## Worksheet: Countries

In [ ]:
# trabalhando com a base Countries
countries_df = pd.read_excel(data, data.sheet_names[0], decimal=',')

In [ ]:
# visualização das primeiras linhas
countries_df.head()

A primeira coisa que iremos fazer é analisar os dados faltantes. Para isso, iremos utilizar a biblioteca `missingno`.

Essa análise inicial é muito importante pois nos permite identificar se existe padrões nos dados faltantes que possibilitam definir a melhor estratégia para tratamento desses dados.

O primeiro método que iremos utilizar do `missingno` é o `matrix`, pois ele nos permite visualizar a localização dos dados faltantes ao longo do nosso dataframe.

In [ ]:
# visualização de dados faltantes
msno.matrix(countries_df, figsize=figsize)

O segundo método é o `bar`, ele nos mostra a quantidade de dados totais disponíveis para cada coluna do dataframe.

In [ ]:
# visualização de dados faltantes
msno.bar(countries_df, figsize=figsize)

Agora como já temos uma ideia inicial da quantidade de dados faltantes e dos padrões, podemos utilizar o método `heatmap` que nos mostra a matriz de correlação entre as amostras faltantes.

In [ ]:
# visualização de dados faltantes
msno.heatmap(countries_df, figsize=figsize)

O matriz de correlação nos dá uma visualização entre pares de variáveis, se quisermos uma informação mais detalhada, podemos utilizar o dendrograma através do método `dendrogram`.

In [ ]:
# visualização de dados faltantes
msno.dendrogram(countries_df, figsize=figsize)

Para fins de simplificação, iremos remover as amostras com dados faltantes.

In [ ]:
# remoção das amostras com dados faltantes
countries_df.dropna(inplace=True)

In [ ]:
# nomes das colunas
countries_df.columns.values

In [ ]:
# validação da remoção de dados faltantes
msno.bar(countries_df, figsize=figsize)

## Análise das distribuições

O primeiro passo é tentar identificar o que é variável categórica e o que é variável numérica. Isso é importante pois nos ajuda a definir estratégias de agrupamento para visualização.

In [ ]:
# número de valores únicos
countries_df.nunique().plot.bar(rot=90,
                                grid=True,
                                title='# valores únicos')

In [ ]:
# valores únicos
print(f"Valores únicos da coluna {countries_df.columns[1]}:")
print(countries_df[countries_df.columns[2]].unique())

print(f"\nValores únicos da coluna {countries_df.columns[2]}:")
print(countries_df[countries_df.columns[3]].unique())

In [ ]:
# visualização da distribuição do GDP
_, ax = plt.subplots(ncols=2, figsize=(15, 7))
countries_df.hist(column='GDP 2018', grid=True, ax=ax[0])
countries_df.boxplot(column='GDP 2018', ax=ax[1]);

In [ ]:
# separação por região mundial
countries_df.boxplot(column='GDP 2018', by=countries_df.columns[2], rot=30);

In [ ]:
# visualização utilizando o seaborn
sns.boxplot(data=countries_df,
            y='GDP 2018',
            x='World Bank region')
plt.xticks(rotation=30);

In [ ]:
# separação por grupo financeiro
countries_df.boxplot(column='GDP 2018', by=countries_df.columns[3], rot=30);

In [ ]:
# visualização utilizando o seaborn
sns.boxplot(data=countries_df,
            y='GDP 2018',
            x=countries_df.columns[3])
plt.xticks(rotation=30)
plt.show()

In [ ]:
# visualização da distribuição do PIB por região
countries_df.hist(column='GDP 2018',
                  by=countries_df.columns[2],
                  figsize=(17, 15),
                  xrot=45)
plt.show()

# Worksheet: Mitigation
## Análise dos dados

In [ ]:
# mitigation worksheet
mitigation_df = pd.read_excel(data, data.sheet_names[1])

In [ ]:
# primeiras colunas
mitigation_df.head()

In [ ]:
# pair plot
sns.pairplot(mitigation_df,
             hue='Strategy',
             vars=['R0', 'Social_distance', 'total_infected']);

In [ ]:
# nomes das colunas disponíveis
print(mitigation_df.columns)

In [ ]:
# slice nos dados do Brasil
br_mitigation_df = mitigation_df.loc[mitigation_df['Country']=='Brazil', :]
br_mitigation_df

In [ ]:
# criando novas features com os valores percentuas e em milhões
perc_names = list(br_mitigation_df.columns.values[5:9]+'_perc')
mil_names = list(br_mitigation_df.columns.values[5:9]+'_million')
br_mitigation_df[perc_names] = br_mitigation_df.loc[:,'total_infected':
                                                        'total_critical'].div(br_mitigation_df['total_pop'],
                                                                                              axis=0)*100
br_mitigation_df[mil_names] = br_mitigation_df.loc[:,'total_infected':
                                                       'total_critical']*1e-6

In [ ]:
# visualizando as 5 primeias linhas do dataframe criado
br_mitigation_df

In [ ]:
# dataframe: no social distancing
no_social_distance = br_mitigation_df.loc[br_mitigation_df['Strategy']==
                                          'Unmitigated', :].drop(["Social_distance",
                                                                  "Country"], axis=1)
no_social_distance

In [ ]:
# plot
fig, ax = plt.subplots(ncols=2, figsize=(15, 5))

# plot percentual
no_social_distance.plot(x="R0",
                        y=['total_deaths_perc',
                           'total_hospital_perc',
                           'total_critical_perc'],
                        style='o-',
                        ax=ax[0],
                        label=['# percentual de mortes',
                               '# percentual de hospitalizações',
                               '# percentual de hospitalizações em UTIs'])
ax[0].set_ylabel("% da população")
ax[0].set_xlabel("Número básico de reprodução, R0")
ax[0].set_xlim([2.3, 3.4])
ax[0].grid()

# plot valores absolutos (em milhões)
no_social_distance.plot(x='R0',
                        y=['total_deaths_million',
                           'total_hospital_million',
                           'total_critical_million'],
                        style='o-',
                        ax=ax[1],
                        label=['# de mortes',
                               '# de hospitalizações',
                               '# de hospitalizações em UTIs'])
ax[1].set_ylabel("Número total, em milhões")
ax[1].set_xlabel("Número básico de reprodução, R0")
ax[1].set_xlim([2.3, 3.4])
ax[1].grid()
plt.suptitle("Simulação do impacto do Covid19 no Brasil quando o \
distanciamento social não é adotado")
plt.show()

In [ ]:
# plot
fig, ax0 = plt.subplots(ncols=1)

# plot percentual
no_social_distance.plot(x="R0",
                        y=['total_deaths_perc',
                           'total_hospital_perc',
                           'total_critical_perc'],
                        style='o-',
                        ax=ax0,
                        label=['# de mortes',
                               '# de hospitalizações',
                               '# de hospitalizações em UTIs'],
                        title="Simulação do impacto do Covid19 no Brasil quando o \
distanciamento social não é adotado")
ax0.set_ylabel("% da população")
ax0.set_xlabel("Número básico de reprodução, R0")
ax0.set_xlim([2.3, 3.4])
ax0.grid()

# twin y-axis
ax1 = ax0.twinx()

# plot valores absolutos (em milhões)
no_social_distance.plot(x='R0',
                        y=['total_deaths_million',
                           'total_hospital_million',
                           'total_critical_million'],
                        style='o-',
                        legend=False,
                        ax=ax1)
ax1.set_ylabel("Número total, em milhões");

In [ ]:
# dataframe com todas as estratégias de mitigação no Brasil
social_distance = br_mitigation_df.drop(["Country"], axis=1)

In [ ]:
# features a serem plotadas em percentual e por milhão
y = ['total_deaths',
     'total_hospital',
     'total_critical']

# número de features
n = len(y)  

# instanciando o objeto para plot
# grid de plot com 2 linhas e 3 (len(y)) colunas compartilhando o eixo x e 
# compartilhando o eixo y nas linhas
fig, ax = plt.subplots(ncols=len(y),
                       nrows=2,
                       figsize=(15, 7),
                       sharex=True,
                       sharey='row')

# laço nas i features
for i in range(n):
    # colunas percentual e por milhão: concatenando as strings
    column = [y[i]+'_perc', y[i]+'_million']

    # dataframe com as colunas [y[i]+'_perc', y[i]+'_million', 'R0', 'Strategy']
    df = social_distance.loc[:, column+['R0', 'Strategy']]

    # laço em cada uma das estratégias
    for s in df['Strategy'].unique():
        # plot percentual (column[0]) da estratégia s na linha 0, coluna i
        df.loc[df['Strategy']==s, :].plot(x='R0',
                                          y=[column[0]],
                                          style='o-',
                                          ax=ax[0][i],
                                          label=[s])
        
        # plot absoluto (column[1]) da estratégia s na linha 1, coluna i
        df.loc[df['Strategy']==s, :].plot(x='R0',
                                          y=[column[1]],
                                          style='o-',
                                          ax=ax[1][i],
                                          label=[s])
    
    # configuração dos plots da linha 0, coluna i
    ax[0][i].grid()
    ax[0][i].set_title(column[0])
    ax[0][i].set_xlim([2.3, 3.4])
    ax[0][i].set_ylabel("% da população")
    ax[0][i].set_xlabel("Número básico de reprodução, R0")

    # configuração dos plots da linha 1, coluna i
    ax[1][i].grid()
    ax[1][i].set_title(column[1])
    ax[1][i].set_xlim([2.3, 3.4])
    ax[1][i].set_ylabel("Total, em milhões")
    ax[1][i].set_xlabel("Número de básico de reprodução, R0")

# Worksheet: Suppression
## Análise dos dados

In [ ]:
# Aba Supression
supression_df = pd.read_excel(data, data.sheet_names[2])

# print nos nomes das colunas
print(supression_df.columns)

In [ ]:
# slice nos dados do Brasil
br_supression_df = supression_df.loc[supression_df['Country']=='Brazil',
                                     ['Country',
                                      'Strategy',
                                      'peak_hospital_bed_demand',
                                      'peak_critical_bed_demand']]

In [ ]:
# por 100.000
c = ['peak_hospital_bed_demand','peak_critical_bed_demand']
br_supression_df[[c+'_per1ht' for c in c]] = br_supression_df.loc[:, c]*1e-5

In [ ]:
# ocupação de leitos hospitalares por 100.000
br_supression_df.sort_values(by='peak_hospital_bed_demand', inplace=True)
br_supression_df.plot.bar(x='Strategy',
                          y=['peak_hospital_bed_demand_per1ht',
                             'peak_critical_bed_demand_per1ht'],
                          figsize=(12, 5),
                          rot=0,
                          grid=True,
                          title='Ocupação de leitos por 100.000 da população');

In [ ]:
# ocupação de leitos hospitalares por 100.000
br_supression_df.plot.bar(x='Strategy',
                          y=['peak_hospital_bed_demand_per1ht',
                             'peak_critical_bed_demand_per1ht'],
                          figsize=(12, 5),
                          rot=0,
                          grid=True,
                          title='Ocupação de leitos por 100.000 da população',
                          stacked=True);

## Usando o Plotly...

O plotly é uma biblioteca de visualização poderosíssima e que nos permite realizar visualizações do tipo:

In [ ]:
# plotly.express: interface alto nível para objetos do plotly
import plotly.express as px

In [ ]:
# exemplo
df = px.data.gapminder()
fig = px.scatter_geo(df,
                     locations="iso_alpha",
                     color="continent",
                     hover_name="country",
                     size="pop",
                     projection="natural earth",
                     animation_frame='year')
fig.show()

In [ ]:
# plotly.graph_objects: manipulação dos objetos de plot
import plotly.graph_objects as go

In [ ]:
# criando os traces do plot

# primeiro traçado: peak_hospital_bed_demand_per1ht
trace_1 = go.Bar(name='peak_hospital_bed_demand_per1ht',
                 x=br_supression_df['Strategy'],
                 y=br_supression_df['peak_hospital_bed_demand_per1ht'],
                 hoverinfo='y')

# segundo traçado: peak_critical_bed_demand_per1ht
trace_2 = go.Bar(name='peak_critical_bed_demand_per1ht',
                 x=br_supression_df['Strategy'],
                 y=br_supression_df['peak_critical_bed_demand_per1ht'],
                 hoverinfo='y')

# objeto do plot: data é uma lista com todos os traçados do plot
fig = go.Figure(data=[trace_1, trace_2])

# visualizando a plot
fig.update_layout(barmode='group')
fig.show()

In [ ]:
# mundando o estilo para stacked
fig.update_layout(barmode='stack',
                  yaxis={'title': 'Occupancy per 100,000 Population',
                         'titlefont_size': 16,
                         'tickfont_size': 14},
                  xaxis={'titlefont_size': 16,
                         'tickfont_size': 14})
fig.show()

In [ ]:
# extract data frame
df = mitigation_df.loc[:, ['Country',
                           'Strategy',
                           'total_pop',
                           'total_deaths']]

In [ ]:
# get perceuntial
df['total_deaths_perc'] = df['total_deaths']/df['total_pop']*100

In [ ]:
# group by strategy
df = df.groupby(by=['Strategy', 'Country'], as_index=False).mean()

In [ ]:
df.head()

In [ ]:
# plot
fig = px.scatter_geo(df,
                     locations="Country",
                     color='Country',
                     locationmode='country names',
                     hover_name="Country",
                     size="total_deaths_perc",
                     animation_frame='Strategy',
                     projection="natural earth")

# configurando o layout do título
fig.update_layout(title={'text': "Impactos do Covid19 no mundo",
                         'y':0.95,
                         'x':0.5,
                         'xanchor': 'center',
                         'yanchor': 'top'})
fig.show()